# 🧬 AOP 411 Ontologizer Notebook with BioPortal Integration
This notebook loads AOP 411, uses BioPortal to suggest ontology terms, and lets users export a structured JSON.

In [15]:
# Step 1: Sample AOP 411 Data Initialization
aop_data = {
    "aop_id": "AOP:411",
    "title": "Oxidative stress leading to decreased lung function",
    "key_events": [
        {
            "id": "KE:1",
            "title": "Oxidative stress",
            "description": "An imbalance between the production of reactive oxygen species and antioxidant defenses."
        },
        {
            "id": "KE:2",
            "title": "Decreased lung function",
            "description": "Impaired ability of lungs to perform normal respiratory functions."
        }
    ],
    "key_event_relationships": [
        {
            "id": "KER:1",
            "source": "KE:1",
            "target": "KE:2",
            "relationship": "leads to"
        }
    ]
}

In [16]:
# Step 2: BioPortal Ontology Suggestion Function
import requests

API_KEY = "3c089dc7-5026-47cb-babe-41c608cfe111"  # Replace with your actual BioPortal API Key
BASE_URL = "https://data.bioontology.org"

def suggest_ontology_terms(text, ontologies="GO,ChEBI"):
    # url = f"{BASE_URL}/search?q={text}&ontologies={ontologies}&apikey={API_KEY}"
    url = f"{BASE_URL}/search?q={text}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json().get('collection', [])
        return [(r.get('prefLabel'), r.get('@id')) for r in results[:5] if r.get('prefLabel') and r.get('@id')]
    else:
        print("Error:", response.status_code)
        return []

In [17]:
# Step 3: Display Suggestions for Each Key Event
from IPython.display import display, Markdown

ontology_map = []
for ke in aop_data["key_events"]:
    print(f"\n🔍 Suggestions for: {ke['title']}")
    suggestions = suggest_ontology_terms(ke['title'])
    for label, iri in suggestions:
        print(f" - {label}: {iri}")
    ontology_map.append((ke, suggestions))


🔍 Suggestions for: Oxidative stress
 - Oxidative stress: http://purl.bioontology.org/ontology/MEDDRA/10080562
 - Oxidative Stress: http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C17741
 - Oxidative Stress: http://purl.bioontology.org/ontology/MESH/D018384
 - oxidative stress: http://purl.bioontology.org/ontology/CSP/4001-0099
 - oxidative stress: http://sbmi.uth.tmc.edu/ontology/ochv#24884

🔍 Suggestions for: Decreased lung function
 - Lung function decreased: http://purl.bioontology.org/ontology/MEDDRA/10025093
 - LUNG FUNCTION DECREASED: http://purl.bioontology.org/ontology/CST/LUNG%20FUNC%20DEC
 - decreased lung function: http://sbmi.uth.tmc.edu/ontology/ochv#C0231797
 - Lung function decreased (NOS): http://purl.bioontology.org/ontology/MEDDRA/10025094
 - Lung function: http://purl.bioontology.org/ontology/MEDDRA/10025091


### ✏️ Running the code below will allow you to manually select one of the suggestions above for each KE and input them below.

In [18]:
# Step 4: Input Final Ontology Mappings
from IPython.display import display
import ipywidgets as widgets

ontology_inputs = []
for ke, suggestions in ontology_map:
    dropdown = widgets.Dropdown(
        options=[("(skip)", "")] + [(f"{label} ({iri})", iri) for label, iri in suggestions],
        description=ke['id'],
        layout=widgets.Layout(width='100%')
    )
    display(dropdown)
    ontology_inputs.append((ke, dropdown))

Dropdown(description='KE:1', layout=Layout(width='100%'), options=(('(skip)', ''), ('Oxidative stress (http://…

Dropdown(description='KE:2', layout=Layout(width='100%'), options=(('(skip)', ''), ('Lung function decreased (…

In [19]:
# Step 5: Export Final Ontologized JSON
import json

structured_aop = {
    "aop_id": aop_data["aop_id"],
    "title": aop_data["title"],
    "key_events": [],
    "key_event_relationships": aop_data["key_event_relationships"]
}

for ke, dropdown in ontology_inputs:
    selected_iri = dropdown.value
    selected_label = dropdown.label if hasattr(dropdown, 'label') else dropdown.description
    structured_aop["key_events"].append({
        "id": ke["id"],
        "title": ke["title"],
        "description": ke["description"],
        "ontology_id": selected_iri,
        "ontology_label": selected_label
    })

filename = "aop_411_ontologized_bioportal.json"
with open(filename, "w") as f:
    json.dump(structured_aop, f, indent=2)

print(f"✅ Ontologized AOP exported to {filename}")

✅ Ontologized AOP exported to aop_411_ontologized_bioportal.json


In [20]:
# Step 6: Download the File
from google.colab import files
files.download("aop_411_ontologized_bioportal.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>